# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 4 Assignment:  Regression Neural Network**

**Student Name: Your Name**

# Assignment Instructions

For this assignment you will use the **reg-30-spring-2018.csv** dataset.  This is a dataset that I generated specifically for this semester.  You can find the CSV file in the **data** directory of the class GitHub repository here: [reg-30-spring-2018.csv](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/data/reg-30-spring-2018.csv).

For this assignment you will train a neural network and return the predictions.  You will submit these predictions to the **submit** function.  See [Assignment #1](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

Complete the following tasks:

* Normalize all numeric to zscores and all text/categorical to dummies.  Do not normalize the *target*.
* Your target (y) is the filed named *target*.
* If you find any missing values (NA's), replace them with the median values for that column.
* No need for any cross validation or holdout.  Just train on the entire data set for 250 steps.
* You might get a warning, such as **"Warning: The mean of column pred differs from the solution file by 2.39"**.  Do not worry about small values, it would be very hard to get exactly the same result as I did.
* Your submitted dataframe will have these columns: id, pred.


# Helpful Functions

You will see these at the top of every module and assignment.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [103]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #4 Sample Code

The following code provides a starting point for this assignment.

In [104]:
import os
import pandas as pd
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics

key = "r7Th7Lhul2aj0C93lQNZkA9QZFuc6RW1YfPO4QM2"  # This is an example key and will not work.
file = '/Users/rebekahgriesenauer/Documents/AI/deep_neural_networks/assignments/assignment_griesenauer_class4.ipynb'

# Begin assignment
path = "../data/"
filename_read = os.path.join(path,"reg-30-spring-2018.csv")
df = pd.read_csv(filename_read)

# Encode the feature vector
ids = df['id']
df.drop('id',1,inplace=True)
#Check to see what columns have null values
missing_value_columns = df.columns[df.isna().any()].tolist()
#replace missing values with median
missing_median(df, 'width')

#find columns with numeric data
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
#remove "target" and "id" from numeric columns list
numeric_columns.remove('target')
#encode numeric columns as z-scores
for column in numeric_columns:
    encode_numeric_zscore(df, column, mean=None, sd=None)

#find columns with non-numeric data
text_columns = df.select_dtypes(include = object).columns.tolist()
#ecode text/category items to dummies
for text_col in text_columns:
    encode_text_dummy(df, text_col)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
x,y = to_xy(df, 'target')


# Save a copy, if you like
#submit_df.to_csv('4.csv',index=False)

# Submit the assignment
#submit(source_file=file,data=submit_df,key=key,no=4)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:76: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [105]:
#train regression neural network
model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(50, activation='relu')) # Hidden 2
model.add(Dense(25, activation='relu')) # Hidden 3
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=250)



Epoch 1/250
 - 0s - loss: 12792.7507
Epoch 2/250
 - 0s - loss: 12743.2110
Epoch 3/250
 - 0s - loss: 12585.7793
Epoch 4/250
 - 0s - loss: 12328.9613
Epoch 5/250
 - 0s - loss: 12163.6996
Epoch 6/250
 - 0s - loss: 12056.7167
Epoch 7/250
 - 0s - loss: 11960.7308
Epoch 8/250
 - 0s - loss: 11829.6353
Epoch 9/250
 - 0s - loss: 11699.2591
Epoch 10/250
 - 0s - loss: 11520.8627
Epoch 11/250
 - 0s - loss: 11403.4342
Epoch 12/250
 - 0s - loss: 11142.1745
Epoch 13/250
 - 0s - loss: 10874.8247
Epoch 14/250
 - 0s - loss: 10693.4436
Epoch 15/250
 - 0s - loss: 10363.1729
Epoch 16/250
 - 0s - loss: 9945.6298
Epoch 17/250
 - 0s - loss: 9615.6477
Epoch 18/250
 - 0s - loss: 9327.3872
Epoch 19/250
 - 0s - loss: 8919.9959
Epoch 20/250
 - 0s - loss: 8730.2381
Epoch 21/250
 - 0s - loss: 8049.3774
Epoch 22/250
 - 0s - loss: 7536.1082
Epoch 23/250
 - 0s - loss: 7054.8576
Epoch 24/250
 - 0s - loss: 6616.9485
Epoch 25/250
 - 0s - loss: 6209.3787
Epoch 26/250
 - 0s - loss: 5814.1028
Epoch 27/250
 - 0s - loss: 5283.

Epoch 228/250
 - 0s - loss: 27.4469
Epoch 229/250
 - 0s - loss: 27.8979
Epoch 230/250
 - 0s - loss: 21.6937
Epoch 231/250
 - 0s - loss: 28.8208
Epoch 232/250
 - 0s - loss: 32.2584
Epoch 233/250
 - 0s - loss: 120.9359
Epoch 234/250
 - 0s - loss: 85.0489
Epoch 235/250
 - 0s - loss: 102.9153
Epoch 236/250
 - 0s - loss: 80.5309
Epoch 237/250
 - 0s - loss: 48.3088
Epoch 238/250
 - 0s - loss: 39.2963
Epoch 239/250
 - 0s - loss: 25.6165
Epoch 240/250
 - 0s - loss: 22.5275
Epoch 241/250
 - 0s - loss: 25.9965
Epoch 242/250
 - 0s - loss: 24.2372
Epoch 243/250
 - 0s - loss: 16.3243
Epoch 244/250
 - 0s - loss: 10.0980
Epoch 245/250
 - 0s - loss: 8.2079
Epoch 246/250
 - 0s - loss: 6.3501
Epoch 247/250
 - 0s - loss: 6.7474
Epoch 248/250
 - 0s - loss: 7.7012
Epoch 249/250
 - 0s - loss: 6.4242
Epoch 250/250
 - 0s - loss: 5.8021


In [106]:
pred = model.predict(x)
print("Shape: {}".format(pred.shape))
print(pred)

Shape: (1233, 1)
[[ -6.969337]
 [-19.538631]
 [-23.936865]
 ...
 [ 12.283845]
 [-92.37664 ]
 [-10.014075]]


In [107]:
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"Final score (RMSE): {score}")

Final score (RMSE): 2.0273001194000244


In [108]:
y.mean()

-4.5841475

In [112]:
submit_df=pd.DataFrame()
submit_df['id']=ids
submit_df['pred']=pred

In [113]:
submit_df

,id,pred
0,1,-6.969337
1,2,-19.538631
2,3,-23.936865
3,4,-19.405777
4,5,-19.058651
5,6,-11.587546
6,7,-17.350067
7,8,-19.527866
8,9,-39.649902
9,10,11.235894


In [114]:
# Submit the assignment
submit(source_file=file,data=submit_df,key=key,no=4)

Success: Submitted Assignment #4 for rebekah.griesenauer:
This is your first submission of this assignment.

